# Strands Agent SDK

In [1]:
%load_ext autoreload
%autoreload 2

## 1. 환경변수 및 lib path 설정

In [2]:
import os, sys
from dotenv import load_dotenv

In [3]:
load_dotenv()
print (f'AWS_DEFAULT_REGION: {os.getenv("AWS_DEFAULT_REGION")}')

module_path = ".."
sys.path.append(os.path.abspath(module_path))

AWS_DEFAULT_REGION: us-west-2


## 2. Utilities

### 2.1 Get llm model
- function name: "get_model"
- script path: ["../src/utils/strands_sdk_utils.py"](../src/utils/strands_sdk_utils.py#L51-L100)

### 2.2 Get system prompt
- function name: "apply_prompt_template"
- script path: ["../src/prompts/template.py"](../src/prompts/template.py#L4-L12)

### 2.3 Create agent
- function name: "get_agent"
- script path: ["../src/utils/strands_sdk_utils.py"](../src/utils/strands_sdk_utils.py#L102-L126)

## 3. Agent definition

- Agent name

In [4]:
agent_name = "toy_agent"

- System prompt

In [5]:
%%writefile ../src/prompts/toy_agent.md

---
CURRENT_TIME: {CURRENT_TIME}
AGENT_NAME: {AGENT_NAME}
---

You are Bedrock-Manus, a friendly AI assistant developed by AWS AIML Specialist SA Dongjin Jang.
You specialize in handling greetings, small talk, and knowledge-based question answering using available tools.

## Available Tools

You have access to the following tools that you should use when appropriate:

### 1. Python REPL Tool (python_repl_tool)
**When to use**: Use this tool when users need to execute Python code or perform data analysis:
- Running Python scripts or code snippets
- Data analysis and calculations
- Testing code functionality
- Mathematical computations

**What it does**: Executes Python code in a REPL environment and returns the output

**Input**: Python code string

### 2. Bash Tool (bash_tool) 
**When to use**: Use this tool when users need to execute system commands or perform file operations:
- Running shell commands
- File system operations (ls, mkdir, etc.)
- System information queries
- Development tasks requiring command line operations

**What it does**: Executes bash commands and returns the output

**Input**: A bash command string

## Tool Usage Guidelines

1. **Assess the user's request** - Determine if the question requires tool usage
2. **Choose the appropriate tool** - Select based on the type of information needed
3. **Use RAG tool for knowledge queries** - When the user asks about topics that might be in your knowledge base
4. **Use Python REPL for code execution** - When the user needs to run Python code or perform calculations
5. **Use Bash tool for system operations** - When the user needs to interact with the system
6. **Provide helpful responses** - Always explain the results in a user-friendly way

## Response Style

- Be friendly and conversational
- Provide clear, helpful answers
- When using tools, explain what you're doing and why
- If a tool doesn't provide the needed information, acknowledge this and offer alternatives
- Always prioritize user experience and clarity

Remember to use tools proactively when they can help answer user questions more accurately or completely.

Overwriting ../src/prompts/toy_agent.md


- Get agent

In [ ]:
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template

In [ ]:
agent = strands_utils.get_agent(
    agent_name=agent_name,
    system_prompts=apply_prompt_template(prompt_name=agent_name, prompt_context={"AGENT_NAME": agent_name}),
    agent_type="claude-sonnet-3-7", # claude-sonnet-3-5-v-2, claude-sonnet-3-7
    enable_reasoning=False,
    prompt_cache_info=(False, None), #(False, None), (True, "default")
    streaming=True,
)

#system_prompts=apply_prompt_template(prompt_name=agent_name, prompt_context={"AGENT_NAME": agent_name})
#print (f'System prompt: \n{system_prompts}')

## 4 Invocation

### 4.1 without streaming

- function name: process_streaming_response_yield
- script path: ["../src/utils/strands_sdk_utils.py"](../src/utils/strands_sdk_utils.py#L180-L200)

In [ ]:
user_input = "안녕 만나서 반가워. 나는 지금 Strands Agents SDK 공부중이야."

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=user_input,
    agent_name=agent_name,
    source=agent_name
):  
    #print (event)
    if event.get("event_type") == "text_chunk":
        full_text += event.get("data", "")

response = {"text": full_text}

print (f'\nResponse: {response['text']}')

### 4.2 with streaming

- function name: process_event_for_display
- script path: ["../src/utils/strands_sdk_utils.py"](../src/utils/strands_sdk_utils.py#L297-L341)

In [ ]:
message = "Strands Agents SDK의 장점은 뭐야?"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    strands_utils.process_event_for_display(event)

    if event.get("event_type") == "text_chunk":
        full_text += event.get("data", "")
    
response = {"text": full_text}

print (f'\nResponse: {response['text']}')

## 5. Tools

In [ ]:
from src.tools import python_repl_tool, bash_tool

In [ ]:
agent = strands_utils.get_agent(
    agent_name=agent_name,
    system_prompts=apply_prompt_template(prompt_name=agent_name, prompt_context={"AGENT_NAME": agent_name}),
    agent_type="claude-sonnet-3-7", # claude-sonnet-3-5-v-2, claude-sonnet-3-7
    enable_reasoning=False,
    prompt_cache_info=(False, None), #(False, None), (True, "default")
    streaming=True,
    tools=[python_repl_tool, bash_tool]
)

In [ ]:
message = "../src/prompts 디렉토리 조회해 주세요"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    #print (event)
    strands_utils.process_event_for_display(event)

In [ ]:
message = "Hello world 를 프린팅하는 파이썬 코드를 작성하고 실행시켜 줄래?"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    strands_utils.process_event_for_display(event)

## 6. built-in utility

In [ ]:
from pprint import pprint

### 6.1 Check agent

- Syetem prompt

In [ ]:
system_prompt = agent.system_prompt
pprint(system_prompt)

- Message history

In [ ]:
agent_messages = agent.messages
pprint(agent_messages)

- observility

In [ ]:
pprint(agent.event_loop_metrics)

- Resume

In [ ]:
from strands import Agent
from botocore.config import Config
from strands.models import BedrockModel
from src.utils.bedrock import bedrock_info

In [ ]:
llm_ = BedrockModel(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-7-Sonnet-CRI"),
    streaming=True,
    max_tokens=8192,
    stop_sequencesb=["\n\nHuman"],
    temperature=0.01,
    cache_prompt=None, # None/ephemeral/defalut
    #cache_tools: Cache point type for tools
    boto_client_config=Config(
        read_timeout=900,
        connect_timeout=900,
        retries=dict(max_attempts=50, mode="standard"),
    )
)


agent_ = Agent(
    model=llm_,
    tools=[python_repl_tool, bash_tool],
    system_prompt=system_prompt,
    messages=agent_messages,
    callback_handler=None # async iterator로 대체 하기 때문에 None 설정
)

In [ ]:
message = "이어서 대화 하는거 맞니?"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent_,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    strands_utils.process_event_for_display(event)


### 4.2 [Conversation management](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/?h=conversa)

As conversations grow, managing this context becomes increasingly important for several reasons:

- **Token Limits**: Language models have fixed context windows (maximum tokens they can process)
- **Performance**: Larger contexts require more processing time and resources
- **Relevance**: Older messages may become less relevant to the current conversation
- **Coherence**: Maintaining logical flow and preserving important information


#### 4.2.1. SlidingWindowConversationManager
고정된 수의 최근 메시지를 유지하는 슬라이딩 윈도우 전략을 구현합니다. Agent 클래스에서 기본적으로 사용하는 대화 매니저입니다.

In [ ]:
from strands.agent.conversation_manager import SlidingWindowConversationManager

In [ ]:
# Create a conversation manager with custom window size
conversation_manager = SlidingWindowConversationManager(
    window_size=3,  # Maximum number of messages to keep
    should_truncate_results=True, # Enable truncating the tool result when a message is too large for the model's context window 
)

In [ ]:
agent.conversation_manager = conversation_manager

In [ ]:
message = "안녕 나는 장동진이야"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    strands_utils.process_event_for_display(event)

print ("\n")
pprint (agent.messages)

#### 3.1.2. SummarizingConversationManager

오래된 메시지를 요약하여 중요한 정보를 보존하면서 컨텍스트 한계 내에서 대화를 관리합니다.

**주요 설정:**

| 파라미터 | 타입 | 기본값 | 설명 |
|---------|------|--------|------|
| `summary_ratio` | `float` | `0.3` | 컨텍스트 축소 시 요약할 메시지 비율 (0.1~0.8 범위) |
| `preserve_recent_messages` | `int` | `10` | 항상 유지할 최근 메시지 수 |
| `summarization_agent` | `Agent` | `None` | 요약 생성용 커스텀 에이전트 (system_prompt와 동시 사용 불가) |
| `summarization_system_prompt` | `str` | `None` | 요약용 커스텀 시스템 프롬프트 (agent와 동시 사용 불가) |

> **기본 요약 방식**: 커스텀 설정이 없을 경우, 주요 토픽, 사용된 도구, 기술적 정보를 3인칭 형태의 구조화된 불릿 포인트로 요약합니다.

In [ ]:
from strands.agent.conversation_manager import SummarizingConversationManager

In [ ]:
# Custom system prompt for technical conversations
custom_system_prompt = """
You are summarizing a technical conversation. Create a concise bullet-point summary that:
- Focuses on code changes, architectural decisions, and technical solutions
- Preserves specific function names, file paths, and configuration details
- Omits conversational elements and focuses on actionable information
- Uses technical terminology appropriate for software development

Format as bullet points without conversational language.
"""

conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
    preserve_recent_messages=3,  # Always keep 10 most recent messages
    summarization_system_prompt=custom_system_prompt
)

In [ ]:
agent.conversation_manager = conversation_manager

In [ ]:
message = "안녕 나는 장동진이야"

full_text = ""
async for event in strands_utils.process_streaming_response_yield(
    agent=agent_,
    message=message,
    agent_name=agent_name,
    source=agent_name
):
    #print (event)
    strands_utils.process_event_for_display(event)

print ("\n")
pprint (agent.messages)